# Spark Runtime Architecture

### Why to learn underlying architecture?


- In previously covered concepts we saw how to run spark application in a standalone mode. When it comes to processing Big Data, the Spark application must be written in a way to optimize the resources allocated to the cluster. The architecture understanding will help in visualizing the parallel processing that occurs inside a spark application.

- Having a complete picture is important for you when debugging the application or searching for a documentation. 

- A delightful treat for all the developers is that writing applications for parallel cluster execution use the same API that on a standalone mode. Means, you can use same `pyspark` script on a standalone mode and cluster mode.


### Spark Runtime Architecture


- Spark can run on a wide variety of cluster managers (Hadoop YARN, Apache Mesos, and Spark’s own built-in Standalone cluster manager) in both on-premise and cloud deployments.

- Spark uses a **Master-Slave architecture** in its cluster mode.

<img src="images/masterSlave.jpeg">


- Single Master and multiple Slaves.

- A Spark application is launched on a set of machines using an external service called a **cluster manager**.

- A distributed application is placed in execution by a master using a Central coordinator called **Driver**.

- Tasks are the smallest unit of work in Spark. One Spark job is divided into multiple tasks.

- Executors on worker nodes are responsible for executing these tasks.

<img src = "images/sparkapp.jpg">

Let's get into more details one by one:

1. The Driver
    - Runs the main () function of the application and is the place where the Spark Context is created.

    - It has two main duties:

        - **Converts User Application into tasks**

            - Translates the RDD’s into the execution graph and splits the graph into multiple stages
            - A Spark program implicitly creates a logical directed acyclic graph (DAG) of operations. When the driver runs, it converts this logical graph into a physical execution plan.
        - **Scheduling tasks on executors**
            - Given a physical execution plan, a Spark driver must coordinate the scheduling of individual tasks on executors.
            - When executors are started they register themselves with the driver, so it has a complete view of the application’s executors at all times.
        - **Exposes the information about the running spark application through a Web UI at port 4040**
  
                        

**How important is it to have a smart driver?**

<img src = "images/driver.jpg">

The above picture explains enough! :)

**2. Executors**
    - Spark executors are worker processes responsible for running the individual tasks in a given Spark job
    - Executors are launched once at the beginning of a Spark application and typically run for the entire lifetime of an application, though Spark applications can continue if executors fail. 
    - Executors have two roles. First, they run the tasks that make up the application and return results to the driver. 
    - Second, they provide in-memory storage for RDDs that are cached by user programs, through a service called the Block Manager that lives within each executor. 

**3. Cluster Manager**

    - Spark depends on a cluster manager to launch executors and, in certain cases, to launch the driver.
    - The cluster manager is a pluggable component in Spark. This allows Spark to run on top of different external managers, such as YARN and Mesos, as well as its built-in Stand‐alone cluster manager.

**How these all are chained together?**

- Whenever you invoke an action, the job gets spawned in the driver program. 
- Then the driver runs a job scheduler to divide the job into smaller stages. 
- tasks are created for every job stage. 
- Finally, tasks are delegated to the executors, which perform the actual work. 
- All this machinery exists within the SparkContext object. 
- **It keeps track of the executors, it spawns jobs, and it runs the scheduler. **

**what is the difference between job stages and tasks?.** 

- job stages are defined on RDD level. That is, they are not immediately executable. 
- Tasks, on the other hand, are bound to a particular partitions and thus are immediately executable.
- The idea behind the job stages is to pipeline computation as much as possible, avoiding the unnecessary data materializations. For example, if you applied two filter transformations in a row, it is not necessary to serialize and deserialize data in between. You can simply pass the data through the next predicate. 



**Let's summarize, what happens when an action is called?** 

The SparkContext is the core of your application. It allows your application to connect to a cluster and allocate resources and executors. Then whenever you invoke an action, the SparkContext spawns a job and runs the job scheduler to divide it into stages. 
That is pipelineable parts of your computations. And only transformations with narrow dependencies are pipelined. 
Then tasks are created for every job stage and scheduled to the executors. 
The driver communicates directly with the executors, exchanging with task information and liveness status. I hope now we have a clear picture of what happens when you invoke an action in Spark

**Important:** A spark application consists of 1 Master and may have multiple workers running. There can be multiple executors running on each worker. Also, each executor may run multiple tasks at a time depending on the resources allocated to it.